In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import Window
from pyspark.sql import SparkSession
from pyspark import *

import pyspark.sql.functions as F
from pyspark.sql.functions import col,isnan,when,count,avg,last
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation




In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/24 15:19:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
wine = spark.read.csv('/Users/krishnapandey/Documents/wines_SPA.csv',header=True,sep= ",",inferSchema=True)

In [4]:
wine.columns

['winery',
 'wine',
 'year',
 'rating',
 'num_reviews',
 'country',
 'region',
 'price',
 'type',
 'body',
 'acidity']

In [5]:
wine.printSchema()

root
 |-- winery: string (nullable = true)
 |-- wine: string (nullable = true)
 |-- year: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- num_reviews: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- price: double (nullable = true)
 |-- type: string (nullable = true)
 |-- body: string (nullable = true)
 |-- acidity: string (nullable = true)



In [6]:
wine.count()

7500

In [7]:
wine.dropDuplicates()

DataFrame[winery: string, wine: string, year: string, rating: double, num_reviews: int, country: string, region: string, price: double, type: string, body: string, acidity: string]

In [8]:
wine.count()

7500

In [38]:
wine.describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
winery,7500,None,None,A Coroa,l'Infernal Combier Fischer Gerin
wine,7500,1537.1166666666666,3201.5497731046858,1194,Zona Zepa Monastrell
year,7500,2013.495839112344,6.940449539306355,1910,NA
rating,7500,4.254933333333889,0.11802903457464531,4.2,4.9
num_reviews,7500,451.10906666666665,723.0018561956379,25,32624
country,7500,None,None,Espana,Espana
region,7500,None,None,Abona,Yecla
price,7500,60.095821871608834,150.3566764526835,4.99,3119.08
type,7500,None,None,Albarino,Verdejo


In [10]:
for i in wine.columns:
    wine.select(i).distinct().show()

+------------------+
|            winery|
+------------------+
|    Altos de Rioja|
|      Torre de Ona|
|      Martin Codax|
|           Beronia|
|             Orben|
|           Comenge|
|            Volver|
|Portal del Priorat|
|       Juan Pinero|
|     Oller del Mas|
|           Can Bas|
| Finca Villacreces|
|   Santa Petronila|
|  Lagar de Sabariz|
|      Amos Baneres|
|       Roig Parals|
|    Castell Miquel|
|  Bodegas Valduero|
|          Valderiz|
|             Ausas|
+------------------+
only showing top 20 rows

+--------------------+
|                wine|
+--------------------+
|Montes Obarenes S...|
| Nun Vinya dels Taus|
|Onomastica Rioja ...|
|Ribera del Duero ...|
|200 Monges Rioja ...|
|Cream Tradicion V...|
|Savinat Sauvignon...|
|                Ekam|
|El Titan del Bendito|
|El Cristo de Sama...|
|Solera India Olor...|
|     Seleccion Rioja|
|La Vinya del Vuit...|
|  Roda Reserva Rioja|
|       Punta Esencia|
|Seleccion Especia...|
|Alaya Tierra (Old...|
|         

+-----------+
|num_reviews|
+-----------+
|       4935|
|        471|
|        148|
|       3997|
|       1645|
|        463|
|        243|
|       5614|
|        897|
|        540|
|        392|
|         31|
|        516|
|       1339|
|        251|
|        137|
|         85|
|       3089|
|       2923|
|       1884|
+-----------+
only showing top 20 rows

+-------+
|country|
+-------+
| Espana|
+-------+

+--------------------+
|              region|
+--------------------+
|              Madrid|
|            Mentrida|
|       Rioja Alavesa|
|       Pla i Llevant|
|  Jerez-Xeres-Sherry|
|  Jerez Palo Cortado|
|            La Palma|
|   Jerez Amontillado|
|             Almansa|
|     Pago Calzadilla|
|           Somontano|
|       Utiel-Requena|
|             Cigales|
|         Extremadura|
|            Priorato|
|    Montilla-Moriles|
|             Galicia|
|              Murcia|
|              Malaga|
|Jerez Pedro Ximen...|
+--------------------+
only showing top 20 rows

+--------

In [11]:
from pyspark.sql.functions import col,isnan,when,count
null_count = wine.select([count(when(col(x).contains('None') | \
                            (col(x) == 'NA' ) | \
                            col(x).isNull() | \
                            isnan(x), x
                           )).alias(x)
                    for x in wine.columns])

null_count.show()

+------+----+----+------+-----------+-------+------+-----+----+----+-------+
|winery|wine|year|rating|num_reviews|country|region|price|type|body|acidity|
+------+----+----+------+-----------+-------+------+-----+----+----+-------+
|     0|   0| 290|     0|          0|      0|     0|    0| 545|1169|   1169|
+------+----+----+------+-----------+-------+------+-----+----+----+-------+



In [12]:
#Filling Missing values
from pyspark.sql.functions import *

new_wine = wine.withColumn('acidity',regexp_replace('acidity','NA', '0'))
new_wine = new_wine.withColumn('body',regexp_replace('body','NA', '1'))
new_wine = new_wine.withColumn('year',regexp_replace('year','NA', '2011'))
new_wine = new_wine.withColumn('type',regexp_replace('type','NA', 'Rioja Red'))

In [13]:
from pyspark.sql.functions import col,isnan,when,count
null_count = new_wine.select([count(when(col(x).contains('None') | \
                            (col(x) == 'NA' ) | \
                            col(x).isNull() | \
                            isnan(x), x
                           )).alias(x)
                    for x in new_wine.columns])

null_count.show()

+------+----+----+------+-----------+-------+------+-----+----+----+-------+
|winery|wine|year|rating|num_reviews|country|region|price|type|body|acidity|
+------+----+----+------+-----------+-------+------+-----+----+----+-------+
|     0|   0|   0|     0|          0|      0|     0|    0|   0|   0|      0|
+------+----+----+------+-----------+-------+------+-----+----+----+-------+



In [14]:
new_wine = new_wine.withColumn("rating",col("rating").cast('string '))

In [15]:
new_wine.printSchema()

root
 |-- winery: string (nullable = true)
 |-- wine: string (nullable = true)
 |-- year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- num_reviews: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- price: double (nullable = true)
 |-- type: string (nullable = true)
 |-- body: string (nullable = true)
 |-- acidity: string (nullable = true)



# String Indexer

In [16]:
columns = ["winery","wine","year","region","num_reviews","type","price"]
str_indexers = [StringIndexer(inputCol = c, outputCol = c+"_index").fit(new_wine) for c in list(set(new_wine.columns)-set(columns))]

pipeline = Pipeline(stages=str_indexers)

wine_1 = pipeline.fit(new_wine).transform(new_wine)

In [42]:
wine_1.toPandas()

,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity,body_index,acidity_index,rating_index,country_index
0,Teso La Monja,Tinto,2013,4.9,58,Espana,Toro,995.00,Toro Red,5,3,1.0,0.0,7.0,0.0
1,Artadi,Vina El Pison,2018,4.9,31,Espana,Vino de Espana,313.50,Tempranillo,4,2,0.0,2.0,7.0,0.0
2,Vega Sicilia,Unico,2009,4.8,1793,Espana,Ribera del Duero,324.95,Ribera Del Duero Red,5,3,1.0,0.0,6.0,0.0
3,Vega Sicilia,Unico,1999,4.8,1705,Espana,Ribera del Duero,692.96,Ribera Del Duero Red,5,3,1.0,0.0,6.0,0.0
4,Vega Sicilia,Unico,1996,4.8,1309,Espana,Ribera del Duero,778.06,Ribera Del Duero Red,5,3,1.0,0.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,Contino,Reserva,2016,4.2,392,Espana,Rioja,19.98,Rioja Red,4,3,0.0,0.0,0.0,0.0
7496,Conreria d'Scala Dei,Les Brugueres,2018,4.2,390,Espana,Priorato,16.76,Priorat Red,4,3,0.0,0.0,0.0,0.0
7497,Mustiguillo,Finca Terrerazo,2017,4.2,390,Espana,El Terrerazo,24.45,Red,4,3,0.0,0.0,0.0,0.0
7498,Matarromera,Gran Reserva,2011,4.2,389,Espana,Ribera del Duero,64.50,Ribera Del Duero Red,5,3,1.0,0.0,0.0,0.0


In [43]:
'''for i in wine_1.columns:
    wine_1.select(i).distinct().show()'''

'for i in wine_1.columns:\n    wine_1.select(i).distinct().show()'

22/06/24 18:10:23 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1027725 ms exceeds timeout 120000 ms
22/06/24 18:10:23 WARN SparkContext: Killing executors is not supported by current scheduler.


In [18]:
wine_1.columns

['winery',
 'wine',
 'year',
 'rating',
 'num_reviews',
 'country',
 'region',
 'price',
 'type',
 'body',
 'acidity',
 'body_index',
 'acidity_index',
 'rating_index',
 'country_index']

In [41]:
wine_2.printSchema()

root
 |-- num_reviews: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- acidity_index: double (nullable = false)
 |-- body_index: double (nullable = false)
 |-- rating_index: double (nullable = false)
 |-- country_index: double (nullable = false)



In [19]:
wine_2 = wine_1[
 'num_reviews',
 'price',
 'acidity_index',
 'body_index',
 'rating_index',
 'country_index']


# One Hot Encoder

In [20]:
columns = ["num_reviews","price","country_index"]
one_hot_encoders = [OneHotEncoder(inputCol=c, outputCol=c+"_encoder").fit(wine_2) for c in list(set(wine_2.columns)-set(columns))]

pipeline = Pipeline(stages=one_hot_encoders)

wine_data3 = pipeline.fit(wine_2).transform(wine_2)

In [44]:
wine_data3.toPandas()

,num_reviews,price,acidity_index,body_index,rating_index,country_index,rating_index_encoder,acidity_index_encoder,body_index_encoder
0,58,995.00,0.0,1.0,7.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)"
1,31,313.50,2.0,0.0,7.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0)","(1.0, 0.0, 0.0, 0.0)"
2,1793,324.95,0.0,1.0,6.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)"
3,1705,692.96,0.0,1.0,6.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)"
4,1309,778.06,0.0,1.0,6.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)"
...,...,...,...,...,...,...,...,...,...
7495,392,19.98,0.0,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)"
7496,390,16.76,0.0,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)"
7497,390,24.45,0.0,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)"
7498,389,64.50,0.0,1.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)"


In [21]:
cols = ["num_reviews",'price']
vector_assemblers = [VectorAssembler(inputCols = [col], outputCol= col + "_vec") for col in cols] 
pipeline = Pipeline(stages = vector_assemblers)
temp_data = pipeline.fit(wine_data3)
temp = temp_data.transform(wine_data3)
temp.toPandas()

,num_reviews,price,acidity_index,body_index,rating_index,country_index,rating_index_encoder,acidity_index_encoder,body_index_encoder,num_reviews_vec,price_vec
0,58,995.00,0.0,1.0,7.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)",[58.0],[995.0]
1,31,313.50,2.0,0.0,7.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0)","(1.0, 0.0, 0.0, 0.0)",[31.0],[313.5]
2,1793,324.95,0.0,1.0,6.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)",[1793.0],[324.95]
3,1705,692.96,0.0,1.0,6.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)",[1705.0],[692.96]
4,1309,778.06,0.0,1.0,6.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)",[1309.0],[778.06]
...,...,...,...,...,...,...,...,...,...,...,...
7495,392,19.98,0.0,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)",[392.0],[19.98]
7496,390,16.76,0.0,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)",[390.0],[16.76]
7497,390,24.45,0.0,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)",[390.0],[24.45]
7498,389,64.50,0.0,1.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)",[389.0],[64.5]


In [22]:
min_max_scalers = [MinMaxScaler(inputCol=col+"_vec", outputCol=col + "_scaled") for col in cols]
pipeline = Pipeline(stages = min_max_scalers)
scalerModel = pipeline.fit(temp)
wine_data4 = scalerModel.transform(temp)

In [23]:
wine_data4.toPandas()

,num_reviews,price,acidity_index,body_index,rating_index,country_index,rating_index_encoder,acidity_index_encoder,body_index_encoder,num_reviews_vec,price_vec,num_reviews_scaled,price_scaled
0,58,995.00,0.0,1.0,7.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)",[58.0],[995.0],[0.0010123009908279395],[0.31791309820846536]
1,31,313.50,2.0,0.0,7.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0)","(1.0, 0.0, 0.0, 0.0)",[31.0],[313.5],[0.00018405472560507988],[0.09906906993696392]
2,1793,324.95,0.0,1.0,6.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)",[1793.0],[324.95],[0.05423479247829688],[0.10274590650880353]
3,1705,692.96,0.0,1.0,6.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)",[1705.0],[692.96],[0.05153532316942237],[0.22092168177541432]
4,1309,778.06,0.0,1.0,6.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)",[1309.0],[778.06],[0.0393877112794871],[0.24824908721327898]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,392,19.98,0.0,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)",[392.0],[19.98],[0.01125801404951072],[0.0048136052586791]
7496,390,16.76,0.0,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)",[390.0],[16.76],[0.011196662474309027],[0.0037795953231923294]
7497,390,24.45,0.0,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)",[390.0],[24.45],[0.011196662474309027],[0.006249016566637445]
7498,389,64.50,0.0,1.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)",[389.0],[64.5],[0.011165986686708181],[0.019109916540626635]


In [24]:
wine_data4.columns

['num_reviews',
 'price',
 'acidity_index',
 'body_index',
 'rating_index',
 'country_index',
 'rating_index_encoder',
 'acidity_index_encoder',
 'body_index_encoder',
 'num_reviews_vec',
 'price_vec',
 'num_reviews_scaled',
 'price_scaled']

In [25]:
wine_data5 = wine_data4[
 'price',
 'acidity_index',
 'body_index',
 'rating_index',
 'rating_index_encoder',
 'acidity_index_encoder',
 'body_index_encoder',
 'num_reviews_vec',
 'price_vec',
 'num_reviews_scaled',
]

In [51]:
wine_data5.columns

['price',
 'acidity_index',
 'body_index',
 'rating_index',
 'rating_index_encoder',
 'acidity_index_encoder',
 'body_index_encoder',
 'num_reviews_vec',
 'price_vec',
 'num_reviews_scaled']

In [50]:
wine_cols = wine_data5
[
'acidity_index',
 'body_index',
 'rating_index',
 'rating_index_encoder',
 'acidity_index_encoder',
 'body_index_encoder',
 'num_reviews_scaled',
 'price_scaled']

['acidity_index',
 'body_index',
 'rating_index',
 'rating_index_encoder',
 'acidity_index_encoder',
 'body_index_encoder',
 'num_reviews_scaled',
 'price_scaled']

In [52]:
'''cols = ["num_reviews",'price']
vector_assemblers = [VectorAssembler(inputCols = [col], outputCol= col + "_vec") for col in cols] 
pipeline = Pipeline(stages = vector_assemblers)
temp_data = pipeline.fit(wine_data3)
temp = temp_data.transform(wine_data3)
temp.toPandas()'''



from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['acidity_index',
 'body_index',
 'rating_index',
 'rating_index_encoder',
 'acidity_index_encoder',
 'body_index_encoder',
 'num_reviews_scaled',
 ], outputCol = 'features')
wine_final = vectorAssembler.transform(wine_data5)
wine_final = wine_final.select(['features', 'price'])
wine_final.show()

+--------------------+-----------+
|            features|      price|
+--------------------+-----------+
|(18,[1,2,10,14,17...|      995.0|
|(18,[0,2,12,13,17...|      313.5|
|(18,[1,2,9,10,14,...|     324.95|
|(18,[1,2,9,10,14,...|     692.96|
|(18,[1,2,9,10,14,...|     778.06|
|(18,[1,2,9,10,14,...|      490.0|
|(18,[1,2,9,10,14,...|      349.0|
|(18,[1,2,9,10,14,...|     810.89|
|(18,[1,2,9,10,14,...|      345.0|
|(18,[1,2,9,10,14,...|      315.0|
|(18,[1,2,9,10,14,...|     514.85|
|(18,[1,2,9,10,14,...|     105.15|
|(18,[1,2,9,10,14,...|      430.0|
|(18,[1,2,9,10,14,...|      925.0|
|(18,[0,1,2,9,14,1...|      350.0|
|(18,[1,2,9,10,14,...|     166.18|
|(18,[1,2,9,10,14,...|     1620.0|
|(18,[1,2,9,10,14,...|     431.36|
|(18,[2,9,10,13,17...|195.6273796|
|(18,[1,2,9,10,14,...|     824.43|
+--------------------+-----------+
only showing top 20 rows



In [46]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['acidity_index',
 'body_index',
 'rating_index',
 'rating_index_encoder',
 'acidity_index_encoder',
 'body_index_encoder',
 'num_reviews_scaled',
 ], outputCol = 'features')
wine_final = vectorAssembler.transform(wine_cols)
wine_final = wine_final.select(['features', 'price'])
wine_final.show()

+--------------------+-----------+
|            features|      price|
+--------------------+-----------+
|(18,[1,2,10,14,17...|      995.0|
|(18,[0,2,12,13,17...|      313.5|
|(18,[1,2,9,10,14,...|     324.95|
|(18,[1,2,9,10,14,...|     692.96|
|(18,[1,2,9,10,14,...|     778.06|
|(18,[1,2,9,10,14,...|      490.0|
|(18,[1,2,9,10,14,...|      349.0|
|(18,[1,2,9,10,14,...|     810.89|
|(18,[1,2,9,10,14,...|      345.0|
|(18,[1,2,9,10,14,...|      315.0|
|(18,[1,2,9,10,14,...|     514.85|
|(18,[1,2,9,10,14,...|     105.15|
|(18,[1,2,9,10,14,...|      430.0|
|(18,[1,2,9,10,14,...|      925.0|
|(18,[0,1,2,9,14,1...|      350.0|
|(18,[1,2,9,10,14,...|     166.18|
|(18,[1,2,9,10,14,...|     1620.0|
|(18,[1,2,9,10,14,...|     431.36|
|(18,[2,9,10,13,17...|195.6273796|
|(18,[1,2,9,10,14,...|     824.43|
+--------------------+-----------+
only showing top 20 rows



In [53]:
train, test = wine_final.randomSplit([0.7, 0.3])

In [54]:
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 5214
Test Dataset Count: 2286


In [31]:
train.show()

+--------------------+-----+
|            features|price|
+--------------------+-----+
|(18,[0,1,2,4,11,1...|56.95|
|(18,[0,1,2,4,11,1...|17.74|
|(18,[0,1,2,4,11,1...|29.75|
|(18,[0,1,2,4,11,1...|29.75|
|(18,[0,1,2,4,11,1...| 49.0|
|(18,[0,1,2,4,11,1...|11.99|
|(18,[0,1,2,4,11,1...| 31.9|
|(18,[0,1,2,4,11,1...| 18.5|
|(18,[0,1,2,4,11,1...| 23.7|
|(18,[0,1,2,4,11,1...| 15.9|
|(18,[0,1,2,4,11,1...|26.75|
|(18,[0,1,2,4,11,1...|21.65|
|(18,[0,1,2,4,11,1...| 53.3|
|(18,[0,1,2,4,11,1...|25.95|
|(18,[0,1,2,4,11,1...|24.77|
|(18,[0,1,2,4,11,1...| 17.7|
|(18,[0,1,2,4,11,1...| 25.8|
|(18,[0,1,2,4,11,1...|11.95|
|(18,[0,1,2,4,11,1...|13.65|
|(18,[0,1,2,4,11,1...| 17.9|
+--------------------+-----+
only showing top 20 rows



In [55]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor


from pyspark.ml.evaluation import RegressionEvaluator

# Linear Regression

In [56]:
lr = LinearRegression(featuresCol = 'features', labelCol='price', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train)

lr_predictions = lr_model.transform(test)
lr_predictions.select("prediction","price","features").show(5)

lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="price",metricName="r2")
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
print("mse: %f" % trainingSummary.meanSquaredError)
print("mae: %f" % trainingSummary.meanAbsoluteError)

+------------------+------+--------------------+
|        prediction| price|            features|
+------------------+------+--------------------+
|  52.4181823077806| 29.75|(18,[0,1,2,4,11,1...|
|52.397607329162824| 11.99|(18,[0,1,2,4,11,1...|
|  52.3907490029569|  38.5|(18,[0,1,2,4,11,1...|
| 52.38389067675098|  18.5|(18,[0,1,2,4,11,1...|
| 52.37703235054506|110.47|(18,[0,1,2,4,11,1...|
+------------------+------+--------------------+
only showing top 5 rows

RMSE: 118.504348
r2: 0.392358
mse: 14043.280610
mae: 37.558319


# Decision Tree Regressor

In [35]:
decision_tree = DecisionTreeRegressor(featuresCol = 'features', labelCol = 'price', maxDepth = 3)
decision_tree_model = decision_tree.fit(train)
pred1 = decision_tree_model.transform(test)

dt_predictions = decision_tree_model.transform(test)
dt_predictions.select("prediction","price","features").show(5)

dt_evaluator_r2 = RegressionEvaluator(predictionCol="prediction", labelCol="price",metricName="r2")
dt_evaluator_rmse = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
dt_evaluator_mse = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mse")
dt_evaluator_mae = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mae")

r2 = dt_evaluator_r2.evaluate(dt_predictions)
print("R Squared (R2) on test data = %g" % r2)

rmse = dt_evaluator_rmse.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

mse = dt_evaluator_mse.evaluate(dt_predictions)
print("Mean Squared Error (MSE) on test data = %g" % mse)

mae = dt_evaluator_mae.evaluate(dt_predictions)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

+-----------------+------+--------------------+
|       prediction| price|            features|
+-----------------+------+--------------------+
|68.96839727065513|  44.8|(18,[0,1,2,4,11,1...|
|68.96839727065513| 28.35|(18,[0,1,2,4,11,1...|
|68.96839727065513|  38.5|(18,[0,1,2,4,11,1...|
|68.96839727065513| 13.75|(18,[0,1,2,4,11,1...|
|68.96839727065513|110.47|(18,[0,1,2,4,11,1...|
+-----------------+------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.402831
Root Mean Squared Error (RMSE) on test data = 127.794
Mean Squared Error (MSE) on test data = 16331.3
Mean Absolute Error (MAE) on test data = 40.6813


# Random Forest Regressor

In [36]:
random_forest = RandomForestRegressor(featuresCol = 'features', labelCol = 'price')
random_forest_Model = random_forest.fit(train)
pred2 = random_forest_Model.transform(test)
rf_predictions = random_forest_Model.transform(test)
rf_predictions.select("prediction","price","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator

rf_evaluator_r2 = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="price",metricName="r2")
rf_evaluator_rmse = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="price",metricName="rmse")
rf_evaluator_mse = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="price",metricName="mse")
rf_evaluator_mae = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="price",metricName="mae")

r2 = rf_evaluator_r2.evaluate(rf_predictions)
print("R Squared (R2) on test data = %g" % r2)
rmse = rf_evaluator_rmse.evaluate(rf_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
mse = rf_evaluator_mse.evaluate(rf_predictions)
print("Mean Squared Error (RMSE) on test data = %g" % mse)
mae = rf_evaluator_mae.evaluate(rf_predictions)
print("Mean Absolute Error (RMSE) on test data = %g" % mae)

+----------------+------+--------------------+
|      prediction| price|            features|
+----------------+------+--------------------+
|50.1136156950344|  44.8|(18,[0,1,2,4,11,1...|
|50.1136156950344| 28.35|(18,[0,1,2,4,11,1...|
|50.1136156950344|  38.5|(18,[0,1,2,4,11,1...|
|50.1136156950344| 13.75|(18,[0,1,2,4,11,1...|
|50.1136156950344|110.47|(18,[0,1,2,4,11,1...|
+----------------+------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.431292
Root Mean Squared Error (RMSE) on test data = 124.712
Mean Squared Error (RMSE) on test data = 15553
Mean Absolute Error (RMSE) on test data = 38.4279


# GBT Regressor

In [37]:
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'price', maxIter=10)
gbt_model = gbt.fit(train)
gbt_predictions = gbt_model.transform(test)
gbt_predictions.select('prediction', 'price', 'features').show(5)
gbt_evaluator_rmse = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="price",metricName="rmse")
gbt_evaluator_r2 = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="price",metricName="r2")
gbt_evaluator_mse = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="price",metricName="mse")
gbt_evaluator_mae = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="price",metricName="mae")

rmse = gbt_evaluator_rmse.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

r2 = gbt_evaluator_r2.evaluate(gbt_predictions)
print("R Squared (R2) on test data = %g" % r2)

mse = gbt_evaluator_mse.evaluate(gbt_predictions)
print("Mean Squared Error (MSE) on test data = %g" % mse)

mae = gbt_evaluator_mae.evaluate(gbt_predictions)
print("Mean Absolute Error (RMSE) on test data = %g" % mae)

22/06/24 15:34:48 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/06/24 15:34:48 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


+-----------------+------+--------------------+
|       prediction| price|            features|
+-----------------+------+--------------------+
|35.15767097801779|  44.8|(18,[0,1,2,4,11,1...|
|35.15767097801779| 28.35|(18,[0,1,2,4,11,1...|
|35.15767097801779|  38.5|(18,[0,1,2,4,11,1...|
|35.15767097801779| 13.75|(18,[0,1,2,4,11,1...|
|35.15767097801779|110.47|(18,[0,1,2,4,11,1...|
+-----------------+------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 123.346
R Squared (R2) on test data = 0.443681
Mean Squared Error (MSE) on test data = 15214.2
Mean Absolute Error (RMSE) on test data = 34.7195
